Gans Project (module 50)
# Settings

In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import json
import requests
import pymysql
import sqlalchemy
from IPython.display import JSON
from datetime import datetime, timedelta
from snowmans8key import Lianes_key2 as like
from snowmans8key import Open_weather_key as weakey
from snowmans8_con import snowcon
from sqlroot_con import con
# connection to AWS RDS MySQL instance:
con = snowcon
# connection to mysqlroot server:
# con = con

# Functions

## WebScraping Cities

In [2]:
# function convert_geo_coordinates
def convert_latitude_longitude_to_decimal(input):
    try: # extract all 4 parts from the latitude / longitude string
        degrees, minutes, seconds, direction = re.split("[^\d\w]+", input)
    except: # exception for Hamburg (has only 3 parts)
        degrees, minutes, direction = re.split("[^\d\w]+", input)
        seconds = 0

    # transform parts into decimal
    decimal_degrees = int(degrees) + int(minutes)/60 + int(seconds)/(60*60)

    # change decimal to "-" if direction is South or West
    if (direction == "S" or direction == "W"):
        decimal_degrees *= -1

    return decimal_degrees

In [3]:
def about_(cities):

    city_names = []
    countries = []
    latitudes = []
    longitudes = []
    populations = []

    for i in cities:
        url = (f"https://en.wikipedia.org/wiki/{i}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        city_names.append(soup.select("div.fn.org")[0].get_text().strip())   # strip
        countries.append(soup.select(".infobox-data")[0].get_text())
        # latitudes.append(soup.select("span.latitude")[0].get_text())
        # longitudes.append(soup.select("span.longitude")[0].get_text())
        latitudes.append(convert_latitude_longitude_to_decimal(soup.select("span.latitude")[0].get_text())) #lat as decimal
        longitudes.append(convert_latitude_longitude_to_decimal(soup.select("span.longitude")[0].get_text())) #lon as deciaml
        if soup.select_one('th.infobox-header:-soup-contains("Population")'):
            # populations.append(soup.select_one('th.infobox-headercity:-soup-contains("Population")').parent.find_next_sibling().find(string=re.compile(r'\d+')))
            populations.append(int(soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(string=re.compile(r'\d+')).replace(",", "")))  # pop without ','
            
    cities_dict = {"city_name":city_names,
         "country":countries,
         "longitude":longitudes, 
         "latitude":latitudes,
         "population":populations}
    cities_dict

    cities_df = pd.DataFrame(cities_dict)
    return cities_df

## API-key Weather


In [4]:
def weather_in_(cities_df):

    weather_dict = {"city_name":[], "timedate_wea":[], "descr_wea":[], "temp":[],"wind":[], "feels_like":[], "rain_prob":[]}

    for c in cities_df["city_name"]:
        weather = requests.get(f"https://api.openweathermap.org/data/2.5/forecast?q={c}&appid={weakey}&units=metric")
        weather_json = weather.json()
        
        for entry in weather_json["list"]:
            weather_dict["city_name"].append(weather_json["city"]["name"])
            weather_dict["timedate_wea"].append(entry["dt_txt"])
            weather_dict["descr_wea"].append(entry["weather"][0]["description"])
            weather_dict["temp"].append(entry["main"]["temp"])
            weather_dict["wind"].append(entry["wind"]["speed"])
            weather_dict["feels_like"].append(entry["main"]["feels_like"])
            weather_dict["rain_prob"].append(entry["pop"])
            
    weather_df = pd.DataFrame(weather_dict)
    
    # add city_id to weather_df
    # sql_cities = draw_city_id(con)
    sql_cities = pd.read_sql_table("cities", con=con)
    id_dict = sql_cities.set_index("city_name")["city_id"].to_dict()
    weather_df["city_id"] = int("0")

    for key, value in id_dict.items():
        weather_df.loc[weather_df["city_name"] == key, "city_id"] = value

    # reorder of columns
    reorder = ['city_id', 'city_name', 'timedate_wea', 'descr_wea', 'temp', 'wind', 'feels_like', 'rain_prob']
    weather_df = weather_df[reorder]

    # data type
    weather_df['timedate_wea'] = pd.to_datetime(weather_df['timedate_wea'])

    return weather_df

## rapidAPI Airport

produce a function, which retrieves tomorrows flight information for the major airports in the cities you web scraped

In [ ]:
# def city_ID_wi_GEOpandas_(airports_df):

In [5]:
def city_ID_for_(airports_df):
    
    # sql_cities = draw_city_id(con)
    sql_cities = pd.read_sql_table("cities", con=con)

    # eine Bogensekunde = 71,5 km E und 111 km N bei 50°
    sql_cities.latitude = sql_cities.latitude.round(0)
    sql_cities.longitude = sql_cities.longitude.round(0)
    airports_df.latitude = airports_df.latitude.round(0)
    airports_df.longitude = airports_df.longitude.round(0)

    merge = sql_cities.merge(airports_df, how="inner", on="latitude")

    reorder = ["icao", "iata", "airport_name", "city_id"]
    airports_df = merge[reorder]
        
    return airports_df

In [6]:
def airports_in_(cities_df):

    # prepare storage
    airports_dict = {"icao":[], "iata":[], "airport_name":[], "longitude":[], "latitude":[]}

    # iterate over list of cities
    for i in range(len(cities_df["city_name"])): # to iterates over rows of df
        City = cities_df['city_name'][i]
        Longitude = cities_df["longitude"][i]
        Latitude = cities_df["latitude"][i]

        # get json from rapidAPI
        airports = requests.get(f"https://aerodatabox.p.rapidapi.com/airports/search/location/{Latitude}/{Longitude}/km/50/15", 
                       headers={"X-RapidAPI-Key": likey,"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"}, 
                       params={"withFlightInfoOnly":"true"})
        airports_json = airports.json()
    
        # store data in dictionary
        for ap in airports_json["items"]:
            airports_dict["icao"].append(ap["icao"])
            airports_dict["iata"].append(ap["iata"])
            airports_dict["airport_name"].append(ap["name"])
            airports_dict["longitude"].append(ap["location"]["lon"])
            airports_dict["latitude"].append(ap["location"]["lat"])
        
    airports_df = pd.DataFrame(airports_dict)
    
    # calling to add city_ID (and reorder)
    airports_df = city_ID_for_(airports_df)
    
    return airports_df

## Flight Informations

In [7]:
def flights_to_(airports_df):
    
    # prepare storage
    flights_dict = {"flight_nr":[], "timedate_arrive":[], "timedate_revised":[], "status":[], "icao":[], "data_timestamp":[]}
    
    # timerange of 12 hours ahead of 'now': (YYYY-MM-DDTHH:MM)
    now = datetime.now()
    then12 = now + timedelta(hours=12)
    then_str = then12.strftime("%Y-%m-%dT%H:%M")
    then = datetime.strptime(then_str, "%Y-%m-%dT%H:%M")
    n = pd.Timestamp.now()
    # oder:
    # t = n + pd.Timedelta(hours=12)
    # no = n.strftime('%Y-%m-%d %H:%M:%S')
    # the = t.strftime('%Y-%m-%d %H:%M:%S')
    # now = pd.to_datetime(no)
    # then = pd.to_datetime(the)
    # print (now, then)
    
    # iterate over list of airports/ICAOs
    for icao in airports_df["icao"]:
    
        # get json from rapidapi
        flights = requests.get(f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{now}/{then}",
                        headers={"X-RapidAPI-Key": likey,"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"}, 
                       params={"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"false"})
        flights_json = flights.json()

        # iterate over all arrivals and store data in dictionary 
        for arrival in flights_json["arrivals"]:
            flights_dict["flight_nr"].append(arrival.get("number", None))
            flights_dict["timedate_arrive"].append(arrival.get("movement", {}).get("scheduledTime", {}).get("local", None))
            flights_dict["timedate_revised"].append(arrival.get("movement", {}).get("revisedTime", {}).get("local", None))       # nested .get()
            flights_dict["status"].append(arrival.get("status", None))
            flights_dict["icao"].append(icao)
            # flights_dict["aircraft"].append(arrival["aircraft"]["model"]))
            flights_dict["data_timestamp"].append(now)
    
    flights_df = pd.DataFrame(flights_dict)
        
    # to timedate format       
    flights_df['timedate_arrive'] = pd.to_datetime(flights_df.timedate_arrive)
    flights_df['timedate_revised'] = pd.to_datetime(flights_df.timedate_revised)
    flights_df['data_timestamp'] = pd.to_datetime(flights_df.data_timestamp)
    # If you want to remove the time zone information altogether and have a naive datetime    -grrr- ???
    flights_df['timedate_arrive'] = flights_df['timedate_arrive'].dt.tz_localize(None)
    flights_df['timedate_revised'] = flights_df['timedate_revised'].dt.tz_localize(None)
    flights_df['data_timestamp'] = flights_df['data_timestamp'].dt.tz_localize(None)

    # return result
    return flights_df

# Calling Functions & Push

In [8]:
# input
cities = ["Potsdam", "Freising"]#, "Hanau"] #"Aachen", "Bremen", "Copenhagen", "Dresden", "Bonn", "Erfurt", "Freiburg_im_Breisgau", "Heidelberg", "Karlsruhe", "Kassel", "Kiel", "Koblenz", "Leipzig", "Magdeburg", "Mainz", "Mannheim", "Nürnberg", "Potsdam", "Regensburg", "Rostock", "Trier", "Wiesbaden", "Yaroslavl"]

# call & push:
# --cities
cities_df = about_(cities)
cities_df.to_sql('cities',
              if_exists='append',
              con=con,
              index=False)

# --weather
weather_df = weather_in_(cities_df)
weather_df.to_sql('weather',
              if_exists='append',
              con=con,
              index=False)

# --airports
airports_df = airports_in_(cities_df)
airports_df.to_sql('airports',
              if_exists='append',
              con=con,
              index=False)

# --flights
flights_df = flights_to_(airports_df)
flights_df.to_sql('flights',
              if_exists='append',
              con=con,
              index=False)

381